# CheEMBL化合物数据采集 
## 网址
https://www.ebi.ac.uk/chembl/
## 简介
ChEMBL是由欧洲核生物研究组织（European Bioinformatics Institute, EBI）开发的化合物数据库，其包含了超过100万条化合物的结构、活性、活性指标、注释等信息。ChEMBL的目标是成为一个开放、公开、全面的化合物数据库，为全人类提供有效的化学信息。ChEMBL的开发始于2000年，目前已成为欧洲核生物研究组织（EBI）的重要组成部分。

ChEMBL的化合物数据采集主要包括以下几个方面：
1. 化合物结构采集：ChEMBL提供的化合物结构数据包括分子式、二维结构图、三维结构图、分子量等信息。
2. 化合物活性数据采集：ChEMBL提供的化合物活性数据包括分子活性数据、化学反应数据、化学反应物质数据等。
3. 化合物注释数据采集：ChEMBL提供的化合物注释数据包括化学家注释、结构注释、活性注释等。
4. 化合物相互作用数据采集：ChEMBL提供的化合物相互作用数据包括化合物相互作用数据、化合物相互作用网络数据等。
5. 化合物药物数据采集：ChEMBL提供的化合物药物数据包括药物结构、药物活性数据、药物相互作用数据等。

ChEMBL化合物数据采集的主要流程如下：
1. 注册ChEMBL账号：首先需要注册ChEMBL账号，注册地址为https://www.ebi.ac.uk/chembl/user_signup。
2. 选择化合物数据类型：选择需要采集的化合物数据类型，如化合物结构数据、化合物活性数据、化合物注释数据等。
3. 选择采集目标：选择需要采集的化合物数据目标，如特定分子、特定类型化合物、特定药物等。
4. 选择采集方式：选择采集方式，如API、Web界面、Excel文件等。
5. 采集数据：根据选择的采集方式，采集化合物数据。

ChEMBL化合物数据采集的优点有：
1. 全面性：ChEMBL提供的化合物数据覆盖了化学、生物、药物领域，涵盖了各个领域的最新研究成果。
2. 准确性：ChEMBL的化合物数据采集是基于结构、活性、注释等多方面信息，具有较高的准确性。
3. 开放性：ChEMBL的化合物数据采集是开放的，任何人都可以免费使用。

## 目标
本次项目的目标是利用ChEMBL数据库，采集化合物结构、活性、注释数据，并进行数据分析。
了解 ChEMBL 数据库以及如何从 ChEMBL 中提取数据，即感兴趣目标的（化合物、活性数据）对。这些数据集可用于许多化学信息学任务，例如相似性搜索、聚类或机器学习。

## 理论内容
1. 化学信息学基础
2. ChEMBL 数据库
    ChEMBL 网络服务
    ChEMBL 网络资源客户端
3. 化合物结构数据
4. 化合物活性数据
    复合活性测量
     IC50测量
     pIC50值
5. 化合物注释数据
6. 化合物相互作用数据
7. 化合物药物数据
8. 数据分析方法

## 实践内容
目标：获取具有给定目标的生物活性数据的化合物列表
- 连接到 ChEMBL 数据库
- 获取目标数据（例如：VEGFR2 激酶）
    - 获取并下载目标数据
    - 选择目标 ChEMBL ID

- 获取生物活性数据
    - 获取并下载目标的生物活性数据
    - 预处理和过滤生物活性数据

- 获取化合物数据
    - 获取并下载化合物数据
    - 预处理和过滤化合物数据

- 输出生物活性化合物数据
    - 并生物活性和化合物数据，并添加 pIC50 值
    - 绘制具有最高 pIC50 的分子



####  pIC50 值

- 为了便于比较 IC50 值，IC50 值具有较大的值范围并以不同的单位（M、nM、...）给出，通常使用 pIC50 值
- pIC50 是转换为摩尔单位时 IC50 值的负对数： pIC50=−log10(IC50) ， 在哪里 IC50 以 M 为单位指定
- pIC50 值越高表明药物的效力呈指数级增长
- 请注意，转换可以适应相应的 IC50 单位，例如对于纳米： pIC50=−log10(IC50∗10−9)=9−log10(IC50)

 其他活动措施：
此外，IC50和pIC50，还使用其他生物活性测量，例如平衡常数[KI]和半最大有效浓度[EC50]

In [169]:
# 连接到 ChEMBL 数据库
# 首先，导入 ChEMBL Web 资源客户端以及其他 Python 库。

import math
from pathlib import Path
from zipfile import ZipFile
from tempfile import TemporaryDirectory

import numpy as np
import pandas as pd
from rdkit.Chem import PandasTools
from chembl_webresource_client.new_client import new_client
from tqdm.auto import tqdm

In [170]:
from pathlib import Path
import os

# 获取当前工作目录
HERE = Path(os.getcwd())
DATA = HERE / 'data'
if not DATA.exists():
    DATA.mkdir(parents=True, exist_ok=True)
print(DATA)

/Users/wangyang/Desktop/AI-drug-design/list/01_Compound_data_acquisition/data


In [171]:
# 接下来，我们创建用于 API 访问的资源对象。
targets_api = new_client.target
compounds_api = new_client.molecule
bioactivities_api = new_client.activity

In [172]:
# 查看类型
type(targets_api)

chembl_webresource_client.query_set.QuerySet

In [173]:
'''
获取目标数据（VEGFR2 激酶） 
从UniProt 网站获取感兴趣靶点的 UniProt ID（VEGFR2 激酶： P35968 ）https://www.uniprot.org/uniprotkb?query=VGFR2_HUMAN
使用UniProt ID获取目标信息
如果您对其他目标感兴趣，请选择不同的 UniProt ID。
'''

uniprot_id = "P35968"

## 从ChEMBL获取目标数据

In [174]:
# 从 ChEMBL 获取目标信息，但仅限于指定值
targets = targets_api.get(target_components__accession=uniprot_id).only(
    "target_chembl_id", "organism", "pref_name", "target_type"
)
print(f'目标的类型是 "{type(targets)}"')

目标的类型是 "<class 'chembl_webresource_client.query_set.QuerySet'>"


## 从ChEMBL下载目标数据

查询结果存储在一个QuerySet（查询集）目标中，也就是说，在我们提出要求之前，查询结果不会从 ChEMBL 中获取（此处使用pandas.DataFrame.from_records）。


In [175]:
targets = pd.DataFrame.from_records(targets)
targets

,organism,pref_name,target_chembl_id,target_type
0,Homo sapiens,Vascular endothelial growth factor receptor 2,CHEMBL279,SINGLE PROTEIN
1,Homo sapiens,Vascular endothelial growth factor receptor 2,CHEMBL279,SINGLE PROTEIN
2,Homo sapiens,Vascular endothelial growth factor receptor,CHEMBL2095227,PROTEIN FAMILY
3,Homo sapiens,Vascular endothelial growth factor receptor 2 ...,CHEMBL2111409,SELECTIVITY GROUP
4,Homo sapiens,VEGF-receptor 2 and stem cell growth factor re...,CHEMBL2111428,SELECTIVITY GROUP
5,Homo sapiens,VEGF-receptor 2 and Fibroblast growth factor r...,CHEMBL2111434,SELECTIVITY GROUP
6,Homo sapiens,VEGF-receptor 2 and Fibroblast growth factor r...,CHEMBL2111439,PROTEIN FAMILY
7,Homo sapiens,VEGF-receptor 2 and PDGF-receptor beta (KDR an...,CHEMBL2111440,SELECTIVITY GROUP
8,Homo sapiens,Vascular endothelial growth factor receptor 1 ...,CHEMBL2111480,SELECTIVITY GROUP
9,Homo sapiens,VEGF-receptor 2 and tyrosine-protein kinase SRC,CHEMBL2111336,SELECTIVITY GROUP


## 选择目标（目标 ChEMBL ID）

检查条目后，我们选择第一个条目作为关注目标：

In [176]:
target = targets.iloc[0]
target

organism                                             Homo sapiens
pref_name           Vascular endothelial growth factor receptor 2
target_chembl_id                                        CHEMBL279
target_type                                        SINGLE PROTEIN
Name: 0, dtype: object

## 保存选定的 ChEMBL ID

In [177]:
chembl_id = target.target_chembl_id
print(f"目标 ChEMBL ID 为 {chembl_id}")

目标 ChEMBL ID 为 CHEMBL279


## 获取生物活性数据

- 现在，我们要查询感兴趣目标的生物活性数据。

- 从ChEMBL中获取目标的生物活性数据

- 在这一步中，我们将获取生物活性数据，并对其进行筛选，以便只考虑

    - 人类蛋白质
    - 生物活性类型 IC50、
    -  精确测量（关系式"="）
    - 结合数据（化验类型"B"）

In [178]:
# 使用bioactivities_api接口过滤符合条件的生物活动数据
# 过滤条件：目标ID为chembl_id，类型为"IC50"，关系符为"="，实验类型为"B"
bioactivities = bioactivities_api.filter(
    target_chembl_id=chembl_id,   # 指定目标的ChEMBL ID
    type="IC50",                 # 活动类型为IC50（半数抑制浓度）
    relation="=",                # 关系符为等于
    assay_type="B"                # 实验类型为B
).only(
    # 选择返回的数据字段
    "activity_id",               # 活动ID
    "assay_chembl_id",           # 实验的ChEMBL ID
    "assay_description",         # 实验描述
    "assay_type",                # 实验类型
    "molecule_chembl_id",        # 分子的ChEMBL ID
    "type",                      # 活动类型
    "standard_units",            # 标准单位
    "relation",                  # 活动值的关系符
    "standard_value",            # 标准化后的活动值
    "target_chembl_id",          # 目标的ChEMBL ID
    "target_organism",           # 目标所属的生物体
)

# 打印生物活动对象的长度（数量）和数据类型
print(f"生物活动对象的长度和类型: {len(bioactivities)}, {type(bioactivities)}")

# 这段代码的作用是通过API过滤并获取符合条件的生物活动数据，然后输出获取到的数据数量和其数据类型。

生物活动对象的长度和类型: 9149, <class 'chembl_webresource_client.query_set.QuerySet'>


## 生物活性集的每个条目都包含以下信息：

In [179]:
print(f"第一个元素的长度和类型: {len(bioactivities[0])}, {type(bioactivities[0])}")
bioactivities[0]

第一个元素的长度和类型: 13, <class 'dict'>


{'activity_id': 32262,
 'assay_chembl_id': 'CHEMBL816964',
 'assay_description': 'Inhibition of tyrosine phosphorylation in cells expressing human VEGFR2 (Flk-1/KDR) kinase',
 'assay_type': 'B',
 'molecule_chembl_id': 'CHEMBL68920',
 'relation': '=',
 'standard_units': 'nM',
 'standard_value': '16500.0',
 'target_chembl_id': 'CHEMBL279',
 'target_organism': 'Homo sapiens',
 'type': 'IC50',
 'units': 'uM',
 'value': '16.5'}

'''
- **activity_id**: `32262`
  - 这是生物活性数据的唯一标识符，表示该条记录的编号。

- **assay_chembl_id**: `'CHEMBL816964'`
  - 这是实验（assay）的ChEMBL ID，用于唯一标识该生物实验。

- **assay_description**: `'Inhibition of tyrosine phosphorylation in cells expressing human VEGFR22 (Flk-1/KDR) kinase'`
  - 实验的描述，说明实验的内容。在这里，实验是研究一种化合物对人类VEGFR22（Flk-1/KDR）激酶磷酸化的抑制作用。

- **assay_type**: `'B'`
  - 实验类型，可能代表某种特定的实验类别或方法，例如细胞实验或体外实验。

- **molecule_chembl_id**: `'CHEMBL68920'`
  - 这是分子或化合物的ChEMBL ID，表示该条记录中研究的化合物的唯一标识符。

- **relation**: `'='`
  - 表示数值关系，在这里表示该生物活性值是精确的等于给出的值。

- **standard_units**: `'nM'`
  - 这是生物活性标准化后的单位。在该条记录中，单位是纳摩尔（nM）。

- **standard_value**: `'16500.0'`
  - 这是标准化后的生物活性值，表示IC50的值为16500纳摩尔（nM），即16.5微摩尔（uM）。

- **target_chembl_id**: `'CHEMBL279'`
  - 这是目标蛋白（如酶或受体）的ChEMBL ID，在此记录中，目标是编号为CHEMBL279的蛋白。

- **target_organism**: `'Homo sapiens'`
  - 目标生物体，这里表示实验是在人体的细胞或蛋白质上进行的。

- **type**: `'IC50'`
  - 活性类型，IC50表示半数抑制浓度，即能够抑制50%目标生物活性的化合物浓度。

- **units**: `'uM'`
  - 生物活性值的原始单位，在此为微摩尔（uM）。

- **value**: `'16.5'`
  - 实验原始记录的活性值，表示IC50为16.5微摩尔（uM）。

这条记录描述了一个化合物（CHEMBL68920）在细胞实验中抑制人类VEGFR22激酶磷酸化的IC50值为16.5微摩尔。
'''

## 从ChEMBL下载生物活性数据

最后，我们以pandasDataFrame 的形式下载QuerySet。

⚠️下面这一行代码差不多需要等待10分钟样子，请您耐心等待，！❤️

In [180]:
bioactivities_df = pd.DataFrame.from_dict(bioactivities)
print(f"数据框架形状: {bioactivities_df.shape}")
bioactivities_df.head()

数据框架形状: (9149, 13)


,activity_id,assay_chembl_id,assay_description,assay_type,molecule_chembl_id,relation,standard_units,standard_value,target_chembl_id,target_organism,type,units,value
0,32262,CHEMBL816964,Inhibition of tyrosine phosphorylation in cell...,B,CHEMBL68920,=,nM,16500.0,CHEMBL279,Homo sapiens,IC50,uM,16.5
1,32335,CHEMBL817617,Inhibition of Vascular endothelial growth fact...,B,CHEMBL69638,=,nM,7100.0,CHEMBL279,Homo sapiens,IC50,uM,7.1
2,33033,CHEMBL812621,Inhibitory activity towards vascular endotheli...,B,CHEMBL419526,=,nM,1880.0,CHEMBL279,Homo sapiens,IC50,nM,1880.0
3,33408,CHEMBL816964,Inhibition of tyrosine phosphorylation in cell...,B,CHEMBL443268,=,nM,10100.0,CHEMBL279,Homo sapiens,IC50,uM,10.1
4,34276,CHEMBL812621,Inhibitory activity towards vascular endotheli...,B,CHEMBL330621,=,nM,19.0,CHEMBL279,Homo sapiens,IC50,nM,19.0


In [181]:
# 保存生物活性数据
bioactivities_df.to_csv(DATA / "bioactivities.csv", index=False)

## 如果使用单位和数值列，我们需要将所有不同单位的数值转换为 nM：

In [182]:
bioactivities_df["units"].unique()

array(['uM', 'nM', 'uM l-1', 'M', 'um', "10'-7g/ml", "10'-6g/ml",
       'ug ml-1', 'microM', 'umol/L', 'nmol/L', "10'-9M", "10'-8M",
       '10^-9M', '10^-7M', '10^-5M', '10^-8M'], dtype=object)

In [183]:
bioactivities_df.drop(["units", "value"], axis=1, inplace=True)
bioactivities_df.head()

,activity_id,assay_chembl_id,assay_description,assay_type,molecule_chembl_id,relation,standard_units,standard_value,target_chembl_id,target_organism,type
0,32262,CHEMBL816964,Inhibition of tyrosine phosphorylation in cell...,B,CHEMBL68920,=,nM,16500.0,CHEMBL279,Homo sapiens,IC50
1,32335,CHEMBL817617,Inhibition of Vascular endothelial growth fact...,B,CHEMBL69638,=,nM,7100.0,CHEMBL279,Homo sapiens,IC50
2,33033,CHEMBL812621,Inhibitory activity towards vascular endotheli...,B,CHEMBL419526,=,nM,1880.0,CHEMBL279,Homo sapiens,IC50
3,33408,CHEMBL816964,Inhibition of tyrosine phosphorylation in cell...,B,CHEMBL443268,=,nM,10100.0,CHEMBL279,Homo sapiens,IC50
4,34276,CHEMBL812621,Inhibitory activity towards vascular endotheli...,B,CHEMBL330621,=,nM,19.0,CHEMBL279,Homo sapiens,IC50


## 预处理和过滤生物活性数据
- 将standard_value的数据类型从对象转换为浮点数
- 删除缺失值的条目
- 只保留标准单位 == nM 的条目
- 删除重复的分子
- 重置数据帧索引
- 重新命名列

## 1.将 "standard_value "的数据类型从 "object "转换为 "float"。

字段standard_value保存的是标准化值（此处为 IC50）。为了在以后的计算中使用这些值，请将其转换为浮点数。

In [184]:
bioactivities_df.dtypes

activity_id            int64
assay_chembl_id       object
assay_description     object
assay_type            object
molecule_chembl_id    object
relation              object
standard_units        object
standard_value        object
target_chembl_id      object
target_organism       object
type                  object
dtype: object

In [185]:
bioactivities_df = bioactivities_df.astype({"standard_value": "float64"})
bioactivities_df.dtypes

activity_id             int64
assay_chembl_id        object
assay_description      object
assay_type             object
molecule_chembl_id     object
relation               object
standard_units         object
standard_value        float64
target_chembl_id       object
target_organism        object
type                   object
dtype: object

## 2.删除缺失值的条目

使用参数inplace=True可直接在当前DataFrame中丢弃数值。

In [186]:
bioactivities_df.dropna(axis=0, how="any", inplace=True)
print(f"数据框架形状: {bioactivities_df.shape}")

数据框架形状: (9148, 11)


## 3.只保留 "标准单位 == nM "的条目

我们只想保留nM 单位的生物活性条目，因此删除了所有其他单位的条目。

In [187]:
print(f"下载数据的单位: {bioactivities_df['standard_units'].unique()}")
print(
    f"非 NM 条目的数量:\
    {bioactivities_df[bioactivities_df['standard_units'] != 'nM'].shape[0]}"
)

下载数据的单位: ['nM' 'ug.mL-1']
非 NM 条目的数量:    25


In [188]:
bioactivities_df = bioactivities_df[bioactivities_df["standard_units"] == "nM"]
print(f"过滤后的单位: {bioactivities_df['standard_units'].unique()}")

过滤后的单位: ['nM']


In [189]:
print(f"数据框架形状: {bioactivities_df.shape}")

数据框架形状: (9123, 11)


In [190]:
# 保存生物活性数据只有nM的数据
bioactivities_df.to_csv(DATA / "bioactivities_only_nM.csv", index=False)

## 4.删除重复的分子

有时，同一个分子（molecule_chembl_id）会被测试多次，在这种情况下，我们只保留第一次测试的结果。

请注意，其他选择可以是保留最佳值或相应化合物所有检测结果的平均值。

In [191]:
bioactivities_df.drop_duplicates("molecule_chembl_id", keep="first", inplace=True)
print(f"数据框架形状: {bioactivities_df.shape}")

数据框架形状: (7504, 11)


## 5.重置 "DataFrame "索引

由于我们删除了一些记录，但我们希望稍后遍历该索引，因此我们重置该索引为连续索引。

In [192]:
bioactivities_df.reset_index(drop=True, inplace=True)
bioactivities_df.head()

,activity_id,assay_chembl_id,assay_description,assay_type,molecule_chembl_id,relation,standard_units,standard_value,target_chembl_id,target_organism,type
0,32262,CHEMBL816964,Inhibition of tyrosine phosphorylation in cell...,B,CHEMBL68920,=,nM,16500.0,CHEMBL279,Homo sapiens,IC50
1,32335,CHEMBL817617,Inhibition of Vascular endothelial growth fact...,B,CHEMBL69638,=,nM,7100.0,CHEMBL279,Homo sapiens,IC50
2,33033,CHEMBL812621,Inhibitory activity towards vascular endotheli...,B,CHEMBL419526,=,nM,1880.0,CHEMBL279,Homo sapiens,IC50
3,33408,CHEMBL816964,Inhibition of tyrosine phosphorylation in cell...,B,CHEMBL443268,=,nM,10100.0,CHEMBL279,Homo sapiens,IC50
4,34276,CHEMBL812621,Inhibitory activity towards vascular endotheli...,B,CHEMBL330621,=,nM,19.0,CHEMBL279,Homo sapiens,IC50


## 6.重新命名列

In [193]:
bioactivities_df.rename(
    columns={"standard_value": "IC50", "standard_units": "units"}, inplace=True
)
bioactivities_df.head()

,activity_id,assay_chembl_id,assay_description,assay_type,molecule_chembl_id,relation,units,IC50,target_chembl_id,target_organism,type
0,32262,CHEMBL816964,Inhibition of tyrosine phosphorylation in cell...,B,CHEMBL68920,=,nM,16500.0,CHEMBL279,Homo sapiens,IC50
1,32335,CHEMBL817617,Inhibition of Vascular endothelial growth fact...,B,CHEMBL69638,=,nM,7100.0,CHEMBL279,Homo sapiens,IC50
2,33033,CHEMBL812621,Inhibitory activity towards vascular endotheli...,B,CHEMBL419526,=,nM,1880.0,CHEMBL279,Homo sapiens,IC50
3,33408,CHEMBL816964,Inhibition of tyrosine phosphorylation in cell...,B,CHEMBL443268,=,nM,10100.0,CHEMBL279,Homo sapiens,IC50
4,34276,CHEMBL812621,Inhibitory activity towards vascular endotheli...,B,CHEMBL330621,=,nM,19.0,CHEMBL279,Homo sapiens,IC50


In [194]:
# 保存生物活性数据过滤好的数据
bioactivities_df.to_csv(DATA / "bioactivities_filtration.csv", index=False)

In [195]:
print(f"数据框架形状： {bioactivities_df.shape}")

数据框架形状： (7504, 11)


现在，我们有了一组7504 个分子 ID，以及各自针对目标激酶的 IC50 值。

## 获取复合数据

我们有一个DataFrame，其中包含针对VEGFR2测试的所有分子（以及各自测得的生物活性）。

现在，我们要获取与各自生物活性 ChEMBL ID 相关联的分子结构。

## 从ChEMBL获取化合物数据

让我们来看看 ChEMBL 中定义了生物活性数据的化合物：我们从 ChEMBL 中获取与我们过滤的生物活性数据相关联的化合物 ID 和结构。

In [196]:
compounds_provider = compounds_api.filter(
    molecule_chembl_id__in=list(bioactivities_df["molecule_chembl_id"])
).only("molecule_chembl_id", "molecule_structures")

## 从ChEMBL下载化合物数据¶

同样，我们要将QuerySet对象导出到pandas.DataFrame 中。考虑到数据量，这可能需要一些时间。因此，我们将首先通过tqdm 获取记录列表，这样就能得到一个漂亮的进度条和一些 ETA。然后，我们可以将化合物列表传递给 DataFrame。

❤️这里需要耐心等待，需要30分钟左右！！！！！❤️

In [197]:
compounds = list(tqdm(compounds_provider))

  0%|          | 0/7504 [00:00<?, ?it/s]

In [198]:
# 保存从ChEMBL下载化合物数据
bioactivities_df.to_csv(DATA / "bioactivities_compounds.csv", index=False)

In [199]:
# # 读取bioactivities_compounds.csv文件
# compounds = pd.read_csv(DATA / "bioactivities_compounds.csv")
# compounds

In [200]:
compounds_df = pd.DataFrame.from_records(
    compounds,
)
print(f"数据框架形状: {compounds_df.shape}")

数据框架形状: (7504, 2)


In [201]:
# 查看前面几行：
compounds_df.head()

,molecule_chembl_id,molecule_structures
0,CHEMBL6246,{'canonical_smiles': 'O=c1oc2c(O)c(O)cc3c(=O)o...
1,CHEMBL7724,{'canonical_smiles': 'Cc1cc2ncc(-c3ccccc3)nc2c...
2,CHEMBL50,{'canonical_smiles': 'O=c1c(O)c(-c2ccc(O)c(O)c...
3,CHEMBL13976,{'canonical_smiles': 'Cc1[nH]c(/C=C2\C(=O)Nc3c...
4,CHEMBL13354,{'canonical_smiles': 'Cc1[nH]c(/C=C2\C(=O)Nc3c...


## 预处理和过滤复合数据

- 删除缺失的条目
- 删除重复的分子（按 molecule_chembl_id 计）
- 使用标准 SMILES 获取分子

## 1.删除缺少分子结构条目的条目

In [202]:
compounds_df.dropna(axis=0, how="any", inplace=True)
print(f"数据框架形状: {compounds_df.shape}")

数据框架形状: (7504, 2)


## 2.删除重复的分子

In [203]:
compounds_df.drop_duplicates("molecule_chembl_id", keep="first", inplace=True)
print(f"数据框架形状: {compounds_df.shape}")

数据框架形状: (7504, 2)


## 3.用标准 SMILES 获取分子

In [204]:
# 到目前为止，我们有多种不同的分子结构表示法。我们只想保留标准的 SMILES。
compounds_df.iloc[0].molecule_structures.keys()

dict_keys(['canonical_smiles', 'molfile', 'standard_inchi', 'standard_inchi_key'])

In [205]:
# 初始化一个空列表，用于存储化合物的canonical SMILES字符串
canonical_smiles = []

# 使用iterrows()方法迭代compounds_df数据框中的每一行数据
for i, compounds in compounds_df.iterrows():
    try:
        # 尝试从当前化合物的'molecule_structures'字段中提取'canonical_smiles'
        canonical_smiles.append(compounds["molecule_structures"]["canonical_smiles"])
    except KeyError:
        # 如果'molecule_structures'字段中没有'canonical_smiles'键，则添加None
        canonical_smiles.append(None)

# 将提取到的canonical_smiles列表作为新列添加到数据框中
compounds_df["smiles"] = canonical_smiles

# 删除不再需要的'molecule_structures'列
compounds_df.drop("molecule_structures", axis=1, inplace=True)

# 输出数据框的形状，显示行数和列数
print(f"数据框架形状: {compounds_df.shape}")


数据框架形状: (7504, 2)


## 合理性检查：删除所有没有规范 SMILES 字符串的分子。

In [206]:
compounds_df.dropna(axis=0, how="any", inplace=True)
print(f"数据框架形状: {compounds_df.shape}")

数据框架形状: (7504, 2)


## 输出（生物活性-化合物）数据

- 化合物和生物活性数据摘要

In [207]:
print(f"生物活性筛选: {bioactivities_df.shape[0]}")
bioactivities_df.columns

生物活性筛选: 7504


Index(['activity_id', 'assay_chembl_id', 'assay_description', 'assay_type',
       'molecule_chembl_id', 'relation', 'units', 'IC50', 'target_chembl_id',
       'target_organism', 'type'],
      dtype='object')

In [208]:
print(f"化合物过滤: {compounds_df.shape[0]}")
compounds_df.columns

化合物过滤: 7504


Index(['molecule_chembl_id', 'smiles'], dtype='object')

## 合并两个数据集

- 根据化合物的 ChEMBL ID（mole_chembl_id），将生物活性数据集（bioactivities_ df）和化合物数据集（compounds_ df）中的相关值合并到输出数据集（output_ df）中，保留以下列：
    - ChEMBL IDs: molecule_chembl_id
    - SMILES: smiles
    - units: units
    - IC50: IC50

In [209]:
# 合并数据
output_df = pd.merge(
    bioactivities_df[["molecule_chembl_id", "IC50", "units"]],
    compounds_df,
    on="molecule_chembl_id",
)

# 重置行指数
output_df.reset_index(drop=True, inplace=True)

print(f"数据集包括 {output_df.shape[0]} 条目.")

数据集包括 7504 条目.


In [210]:
output_df.dtypes

molecule_chembl_id     object
IC50                  float64
units                  object
smiles                 object
dtype: object

In [211]:
output_df.head(10)

,molecule_chembl_id,IC50,units,smiles
0,CHEMBL68920,16500.0,nM,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...
1,CHEMBL69638,7100.0,nM,Nc1ncnc2c1c(-c1cccc(Oc3ccccc3)c1)cn2C1CCCC1
2,CHEMBL419526,1880.0,nM,c1ccc(-c2cnc(Nc3ccccn3)o2)cc1
3,CHEMBL443268,10100.0,nM,Cc1cc(C(=O)NCCN2CCOCC2)[nH]c1/C=C1\C(=O)N(C)c2...
4,CHEMBL330621,19.0,nM,Cc1ccc(Nc2ncc(-c3ccccc3)s2)nc1
5,CHEMBL304271,360.0,nM,CCN(CC)CC(O)CNC(=O)c1cc(C)c(/C=C2\C(=O)Nc3ncnc...
6,CHEMBL98896,23.0,nM,c1ccc(-c2cnc(Nc3cnccn3)s2)cc1
7,CHEMBL98653,240.0,nM,c1ccc(-c2cnc(Nc3cccnc3)s2)cc1
8,CHEMBL319065,6.0,nM,Cc1ccnc(Nc2ncc(-c3ccccc3)s2)c1
9,CHEMBL328644,124.0,nM,c1ccc(Nc2ncc(-c3ccccc3)s2)cc1


## 添加 pIC50值

- 正如您所看到的，低 IC50 值很难读取（值分布在多个刻度上），因此我们将 IC50 值转换为 pIC50。

In [212]:
def convert_ic50_to_pic50(IC50_value):
    pIC50_value = 9 - math.log10(IC50_value)
    return pIC50_value

In [213]:
# 对化合物 DataFrame 的每一行进行转换
output_df["pIC50"] = output_df.apply(lambda x: convert_ic50_to_pic50(x.IC50), axis=1)

In [214]:
output_df.head()

,molecule_chembl_id,IC50,units,smiles,pIC50
0,CHEMBL68920,16500.0,nM,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,4.782516
1,CHEMBL69638,7100.0,nM,Nc1ncnc2c1c(-c1cccc(Oc3ccccc3)c1)cn2C1CCCC1,5.148742
2,CHEMBL419526,1880.0,nM,c1ccc(-c2cnc(Nc3ccccn3)o2)cc1,5.725842
3,CHEMBL443268,10100.0,nM,Cc1cc(C(=O)NCCN2CCOCC2)[nH]c1/C=C1\C(=O)N(C)c2...,4.995679
4,CHEMBL330621,19.0,nM,Cc1ccc(Nc2ncc(-c3ccccc3)s2)nc1,7.721246


## 绘制复合数据

- 让我们来看看收集到的数据集。
- 首先，我们绘制了 pIC50 值的分布图

## 在接下来的步骤中，我们将在DataFrame中添加一列 RDKit 分子对象，并查看 pIC50 值最高的分子的结构。

In [221]:
# 新增 molecule column
PandasTools.AddMoleculeColumnToFrame(output_df, smilesCol="smiles")

In [222]:
# 通过pIC50对分子进行排序
output_df.sort_values(by="pIC50", ascending=False, inplace=True)

# 重置index
output_df.reset_index(drop=True, inplace=True)

In [223]:
# 显示三种最活跃的分子，即 pIC50 值最高的分子。
output_df.drop("smiles", axis=1).head(3)

,molecule_chembl_id,IC50,units,pIC50,ROMol
0,CHEMBL5189340,0.023,nM,10.638272,<rdkit.Chem.rdchem.Mol object at 0x1323c0a50>
1,CHEMBL429743,0.030,nM,10.522879,<rdkit.Chem.rdchem.Mol object at 0x131cb4eb0>
2,CHEMBL5186748,0.120,nM,9.920819,<rdkit.Chem.rdchem.Mol object at 0x1323c0970>


In [224]:
# 准备保存数据集：删除ROMol列
output_df = output_df.drop("ROMol", axis=1)
print(f"数据框架形状: {output_df.shape}")

数据框架形状: (7504, 5)


In [225]:
# 保存最终的数据集
output_df.to_csv(DATA / "bioactivities_compounds_VEGFR2.csv", index=False)